In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits.conversational_retrieval.tool import create_retriever_tool
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
import json



In [2]:
docs = []
with open("final_data.json") as f:
    for line in f:
        row = json.loads(line)
        doc = Document(
            page_content = row["text"],
            metadata={"title": row["title"], "id": row["docid"], "source": row["text"]},
        )
        docs.append(doc)


In [3]:
docs[0]


Document(page_content="There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage", metadata={'title': 'Diabetic neuropathy', 'id': 1, 'source': "There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage"})

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)


In [5]:
docs[0]


Document(page_content="There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage", metadata={'title': 'Diabetic neuropathy', 'id': 1, 'source': "There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage"})

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [7]:
db = FAISS.from_documents(docs, embeddings)
db.save_local("symptomdb.faiss")


In [8]:
# db = FAISS.load_local("symptomdb.faiss", embeddings)


In [9]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import os


In [10]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [11]:
llm = ChatOpenAI(temperature=0)


In [12]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = RetrievalQAWithSourcesChain.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())



# print(result['sources'])


In [13]:
query = "I have a nosebleed and shivers"
result = chain({"question": query}, return_only_outputs=True)
scored_docs = db.similarity_search_with_relevance_scores(query, top_k=3)


In [14]:
result


{'answer': 'Based on the symptoms described, it is possible that you are experiencing acute sinusitis. The symptoms of acute sinusitis include a runny nose, postnasal drip, congestion, and pain or pressure around the eyes, cheeks, nose, or forehead. However, it is important to consult a healthcare professional for an accurate diagnosis and appropriate treatment. As for the shivers, it is not clear from the information provided what may be causing them. It is recommended to seek medical advice for a proper evaluation. \n',
 'sources': "Acute sinusitis symptoms often include:Thick, yellow or greenish mucus from the nose, known as a runny nose, or down the back of the throat, known as postnasal drip. Blocked or stuffy nose, known as congestion. This makes it hard to breathe through the nose. Pain, tenderness, swelling and pressure around the eyes, cheeks, nose or forehead that gets worse when bending over. Shivering is likely the first thing you'll notice as the temperature starts to drop

In [15]:
scored_docs


[(Document(page_content='Acute sinusitis symptoms often include:Thick, yellow or greenish mucus from the nose, known as a runny nose, or down the back of the throat, known as postnasal drip. Blocked or stuffy nose, known as congestion. This makes it hard to breathe through the nose. Pain, tenderness, swelling and pressure around the eyes, cheeks, nose or forehead that gets worse when bending over.', metadata={'title': 'Acute sinusitis', 'id': 257, 'source': 'Acute sinusitis symptoms often include:Thick, yellow or greenish mucus from the nose, known as a runny nose, or down the back of the throat, known as postnasal drip. Blocked or stuffy nose, known as congestion. This makes it hard to breathe through the nose. Pain, tenderness, swelling and pressure around the eyes, cheeks, nose or forehead that gets worse when bending over.'}),
  0.28373782749882503),
 (Document(page_content="Shivering is likely the first thing you'll notice as the temperature starts to drop because it's your body's